<a href="https://colab.research.google.com/github/INLANA01/capstone_project_coref_with_transformers/blob/main/ALBERT_%2B_Neural_Network_Model_and_LateFusion_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [60]:
import os
import gc
import numpy as np
import pandas as pd 
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from tqdm import tqdm, trange
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Some of the libraries in extract_features.py use Tensorflow Version 1, hence need to install older version of TenforFlow and disable v2 behaviour

In [5]:
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

     |████████████████████████████████| 412.3MB 35kB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 512kB 56.1MB/s 
     |████████████████████████████████| 3.8MB 54.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=2be0ada1b1d131f79ae29ea4fe5848de81e9c99fc0cac6dd6321c6da432335cf
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

     |████████████████████████████████| 411.5MB 35kB/s 


Instructions for updating:
non-resource variables are not supported in the long term


# ALBERT tokenizer needs sentencepiece library

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 12.5MB/s 


In [ ]:
!git clone https://github.com/google-research-datasets/gap-coreference.git

## Get the files withour cloning the gitrepo https://stackoverflow.com/questions/4604663/download-single-files-from-github

In [11]:
!wget -q https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv
!wget -q https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv
!wget -q https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv

In [3]:
!wget -q https://raw.githubusercontent.com/google-research/albert/master/modeling.py 
# !wget -q https://raw.githubusercontent.com/google-research/albert/master/create_pretraining_data.py 
!wget -q https://raw.githubusercontent.com/google-research/albert/master/tokenization.py

In [ ]:
!wget -q https://raw.githubusercontent.com/google-research/bert/master/extract_features.py 

In [3]:
import modeling
import tokenization
# import extract_features

# Load the dataset into dataframe using Pandas

In [12]:
val_df = pd.read_table('gap-validation.tsv', index_col='ID').reset_index(drop=True)
test_df  = pd.read_table('gap-validation.tsv', index_col='ID').reset_index(drop=True)
dev_df  = pd.read_table('gap-development.tsv', index_col='ID').reset_index(drop=True)

# Download ALBERT model, to extract features and get embeddings

In [14]:
!wget -q https://storage.googleapis.com/albert_models/albert_base_v2.tar.gz

# Extract Zip file

In [25]:
import tarfile
my_tar = tarfile.open('./albert_base_v2.tar.gz')
my_tar.extractall('.') # specify which folder to extract to
my_tar.close()

# Process the data to extract features and get word embeddings

In [49]:
def count_char(text, offset):   
    # count = 0
    # for pos in range(offset):
    #     if text[pos] != " ": count +=1
    # return count
    return sum([1 if text[pos] != " " else 0 for pos in range(offset)])

def candidate_length(candidate):
    # count = 0
    # for i in range(len(candidate)):
    #     if candidate[i] !=  " ": count += 1
    # return count
    return sum([1 if candidate[i] !=  " " else 0 for i in range(len(candidate))])

def count_token_length_special(token):
    count = 0
    special_token = ["#", " "]
    # for i in range(len(token)):
    #     if token[i] not in special_token: count+=1
    # return count
    return sum([1 if token[i] not in special_token else 0 for i in range(len(token))])

def embed_by_bert(df, path_to_bert='albert_base', embed_size=768, batch_size=8,
                 layers='-1', max_seq_length=256):
    
    text = df['Text']
    text.to_csv('input.txt', index=False, header=False)
    albert_extract()

    bert_output = pd.read_json("output.jsonl", lines=True)
    print(bert_output.head())
    
    os.system("rm input.txt")
    os.system("rm output.jsonl")
    
    index = df.index
    columns = ["emb_A", "emb_B", "emb_P", "label"]
    emb = pd.DataFrame(index = index, columns = columns)
    emb.index.name = "ID"
    
    for i in tqdm(range(len(text))):
        
        features = bert_output.loc[i, "features"]
        P_char_start = count_char(df.loc[i, 'Text'], df.loc[i, 'Pronoun-offset'])
        A_char_start = count_char(df.loc[i, 'Text'], df.loc[i, 'A-offset'])
        B_char_start = count_char(df.loc[i, 'Text'], df.loc[i, 'B-offset'])
        A_length = candidate_length(df.loc[i, 'A'])
        B_length = candidate_length(df.loc[i, 'B'])
        
        emb_A, emb_B, emb_P = np.zeros(embed_size), np.zeros(embed_size), np.zeros(embed_size)
        char_count, cnt_A, cnt_B = 0, 0, 0
        
        for j in range(2, len(features)):
            token = features[j]["token"]
            token_length = count_token_length_special(token)
            if char_count == P_char_start:
                emb_P += np.asarray(features[j]["layers"][0]['values']) 
            if char_count in range(A_char_start, A_char_start + A_length):
                emb_A += np.asarray(features[j]["layers"][0]['values'])
                cnt_A += 1
            if char_count in range(B_char_start, B_char_start + B_length):
                emb_B += np.asarray(features[j]["layers"][0]['values'])
                cnt_B += 1                
            char_count += token_length
        
        if cnt_A > 0:
            emb_A /= cnt_A
        if cnt_B > 0:
            emb_B /= cnt_B
        
        label = "Neither"
        if (df.loc[i,"A-coref"] == True):
            label = "A"
        if (df.loc[i,"B-coref"] == True):
            label = "B"

        emb.iloc[i] = [emb_A, emb_B, emb_P, label]
        
    return emb    

In [17]:
!pip install transformers

     |████████████████████████████████| 2.1MB 11.4MB/s 
     |████████████████████████████████| 3.3MB 49.5MB/s 
     |████████████████████████████████| 901kB 50.2MB/s 


# extract_features.py  from BERT re written for ALBERT

In [50]:
from transformers import AlbertTokenizer, AlbertModel, AlbertConfig
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import codecs
import collections
import json
import re

import modeling
import tokenization
import tensorflow as tf

# flags = tf.flags

# FLAGS = flags.FLAGS

# flags.DEFINE_string("input_file", 'input.txt', "")

# flags.DEFINE_string("output_file", 'output.jsonl', "")

# flags.DEFINE_string("layers", "-1", "")

# flags.DEFINE_string(
#     "bert_config_file", 'albert_base/albert_config.json',
#     "The config json file corresponding to the pre-trained BERT model. "
#     "This specifies the model architecture.")

# flags.DEFINE_integer(
#     "max_seq_length", 128,
#     "The maximum total input sequence length after WordPiece tokenization. "
#     "Sequences longer than this will be truncated, and sequences shorter "
#     "than this will be padded.")

# flags.DEFINE_string(
#     "init_checkpoint", 'albert_base/model.ckpt',
#     "Initial checkpoint (usually from a pre-trained BERT model).")

# flags.DEFINE_string("vocab_file", 'albert_base/30k-clean.vocab',
#                     "The vocabulary file that the BERT model was trained on.")

# flags.DEFINE_bool(
#     "do_lower_case", True,
#     "Whether to lower case the input text. Should be True for uncased "
#     "models and False for cased models.")

# flags.DEFINE_integer("batch_size", 32, "Batch size for predictions.")

# flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")

# flags.DEFINE_string("master", None,
#                     "If using a TPU, the address of the master.")

# flags.DEFINE_integer(
#     "num_tpu_cores", 8,
#     "Only used if `use_tpu` is True. Total number of TPU cores to use.")

# flags.DEFINE_bool(
#     "use_one_hot_embeddings", False,
#     "If True, tf.one_hot will be used for embedding lookups, otherwise "
#     "tf.nn.embedding_lookup will be used. On TPUs, this should be True "
#     "since it is much faster.")

In [51]:

class InputExample(object):

  def __init__(self, unique_id, text_a, text_b):
    self.unique_id = unique_id
    self.text_a = text_a
    self.text_b = text_b


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, unique_id, tokens, input_ids, input_mask, input_type_ids):
    self.unique_id = unique_id
    self.tokens = tokens
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.input_type_ids = input_type_ids


def input_fn_builder(features, seq_length):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_unique_ids = []
  all_input_ids = []
  all_input_mask = []
  all_input_type_ids = []

  for feature in features:
    all_unique_ids.append(feature.unique_id)
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_input_type_ids.append(feature.input_type_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "unique_ids":
            tf.constant(all_unique_ids, shape=[num_examples], dtype=tf.int32),
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_type_ids":
            tf.constant(
                all_input_type_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
    })

    d = d.batch(batch_size=batch_size, drop_remainder=False)
    return d

  return input_fn


def model_fn_builder(bert_config, init_checkpoint, layer_indexes, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    input_type_ids = features["input_type_ids"]

    model = modeling.AlbertModel(
        config=bert_config,
        is_training=False,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=input_type_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    if mode != tf.estimator.ModeKeys.PREDICT:
      raise ValueError("Only PREDICT modes are supported: %s" % (mode))

    tvars = tf.trainable_variables()
    scaffold_fn = None
    (assignment_map,
     initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
         tvars, init_checkpoint)
    if use_tpu:

      def tpu_scaffold():
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        return tf.train.Scaffold()

      scaffold_fn = tpu_scaffold
    else:
      tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    all_layers = model.get_all_encoder_layers()

    predictions = {
        "unique_id": unique_ids,
    }

    for (i, layer_index) in enumerate(layer_indexes):
      predictions["layer_output_%d" % i] = all_layers[layer_index]

    output_spec = tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn


def convert_examples_to_features(examples, seq_length, tokenizer):
  """Loads a data file into a list of `InputBatch`s."""

  features = []
  for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, seq_length - 3)
    else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > seq_length - 2:
        tokens_a = tokens_a[0:(seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    input_type_ids = []
    tokens.append("[CLS]")
    input_type_ids.append(0)
    for token in tokens_a:
      tokens.append(token)
      input_type_ids.append(0)
    tokens.append("[SEP]")
    input_type_ids.append(0)

    if tokens_b:
      for token in tokens_b:
        tokens.append(token)
        input_type_ids.append(1)
      tokens.append("[SEP]")
      input_type_ids.append(1)
    
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < seq_length:
      input_ids.append(0)
      input_mask.append(0)
      input_type_ids.append(0)

    assert len(input_ids) == seq_length
    assert len(input_mask) == seq_length
    assert len(input_type_ids) == seq_length

    if ex_index < 5:
      tf.logging.info("*** Example ***")
      tf.logging.info("unique_id: %s" % (example.unique_id))
      tf.logging.info("tokens: %s" % " ".join(
          [tokenization.printable_text(x) for x in tokens]))
      tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
      tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
      tf.logging.info(
          "input_type_ids: %s" % " ".join([str(x) for x in input_type_ids]))

    features.append(
        InputFeatures(
            unique_id=example.unique_id,
            tokens=tokens,
            input_ids=input_ids,
            input_mask=input_mask,
            input_type_ids=input_type_ids))
  return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()


def read_examples(input_file):
  """Read a list of `InputExample`s from an input file."""
  examples = []
  unique_id = 0
  with tf.gfile.GFile(input_file, "r") as reader:
    while True:
      line = tokenization.convert_to_unicode(reader.readline())
      if not line:
        break
      line = line.strip()
      text_a = None
      text_b = None
      m = re.match(r"^(.*) \|\|\| (.*)$", line)
      if m is None:
        text_a = line
      else:
        text_a = m.group(1)
        text_b = m.group(2)
      examples.append(
          InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b))
      unique_id += 1
  return examples



In [52]:
def albert_extract():
    tf.logging.set_verbosity(tf.logging.INFO)

    #   layer_indexes = [int(x) for x in FLAGS.layers.split(",")]
    layer_indexes = [-1]

    #   bert_config = modeling.AlbertConfig.from_json_file('albert_base/albert_config.json')

    model_path = "albert-base-v2"
    do_lower_case = True
    bert_config = AlbertConfig.from_pretrained(model_path,hidden_act ="gelu")
    tokenizer = AlbertTokenizer.from_pretrained(model_path, do_lower_case=do_lower_case)
    #   tokenizer = tokenization.FullTokenizer(
    #       vocab_file='albert_base/30k-clean.vocab', do_lower_case=True)

    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
    run_config = tf.contrib.tpu.RunConfig(
        master=None,
        tpu_config=tf.contrib.tpu.TPUConfig(
            num_shards=8,
            per_host_input_for_training=is_per_host))

    examples = read_examples('input.txt')

    features = convert_examples_to_features(
        examples=examples, seq_length=512, tokenizer=tokenizer)

    unique_id_to_feature = {}
    for feature in features:
        unique_id_to_feature[feature.unique_id] = feature

    model_fn = model_fn_builder(
        bert_config=bert_config,
        init_checkpoint='albert_base/model.ckpt-best',
        layer_indexes=layer_indexes,
        use_tpu=False,
        use_one_hot_embeddings=False)

    # If TPU is not available, this will fall back to normal Estimator on CPU
    # or GPU.
    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=False,
        model_fn=model_fn,
        config=run_config,
        predict_batch_size=32)

    input_fn = input_fn_builder(
        features=features, seq_length=512)

    with codecs.getwriter("utf-8")(tf.gfile.Open('output.jsonl',
                                                "w")) as writer:
        for result in estimator.predict(input_fn, yield_single_examples=True):
            unique_id = int(result["unique_id"])
            feature = unique_id_to_feature[unique_id]
            output_json = collections.OrderedDict()
            output_json["linex_index"] = unique_id
            all_features = []
            for (i, token) in enumerate(feature.tokens):
                all_layers = []
                for (j, layer_index) in enumerate(layer_indexes):
                    layer_output = result["layer_output_%d" % j]
                    layers = collections.OrderedDict()
                    layers["index"] = layer_index
                    layers["values"] = [
                        round(float(x), 6) for x in layer_output[i:(i + 1)].flat
                    ]
                    all_layers.append(layers)
                    features = collections.OrderedDict()
                    features["token"] = token
                    features["layers"] = all_layers
                    all_features.append(features)
            output_json["features"] = all_features
            writer.write(json.dumps(output_json) + "\n")

In [23]:
from transformers import AlbertTokenizer, AlbertModel, AlbertConfig

# Multi-class Neural Network 
## Detailed explaination of how to train a multiclass neural network model using PyTorch: https://towardsdatascience.com/pytorch-tabular-multiclass-classification-9f8211a123ab

In [54]:
class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [55]:
def featurize(embedding_df):
    
    pronoun_embs, a_embs, b_embs, labels = [], [], [], []
    
    for i in tqdm(range(len(embedding_df))):
        
        pronoun_embs.append(embedding_df.loc[i, "emb_P"])
        a_embs.append(embedding_df.loc[i, "emb_A"])
        b_embs.append(embedding_df.loc[i, "emb_B"])

        label_map = {'A': 0, 'B': 1, 'Neither': 2}
        labels.append(label_map[embedding_df.loc[i, "label"]])

    
    a_embs = np.asarray(a_embs).astype('float')
    b_embs = np.asarray(b_embs).astype('float') 
    pronoun_embs = np.asarray(pronoun_embs).astype('float')
    
    return np.concatenate([a_embs, b_embs, pronoun_embs], axis=1), np.asarray(labels)

In [56]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


In [57]:
def get_class_distribution(obj):
    count_dict = {
        "A": 0,
        "B": 0,
        "N": 0,
    }
    
    for i in obj:
        if i == 0: 
            count_dict['A'] += 1
        elif i == 1: 
            count_dict['B'] += 1
        elif i == 2: 
            count_dict['N'] += 1
            
    return count_dict

In [58]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

# Training the model based on the embeddings from ALBERT.

In [59]:
import time
start = time.time()

val_df = pd.read_table('gap-validation.tsv', index_col='ID').reset_index(drop=True)
test_df  = pd.read_table('gap-test.tsv', index_col='ID').reset_index(drop=True)
dev_df  = pd.read_table('gap-development.tsv', index_col='ID').reset_index(drop=True)

val_bert_emb = embed_by_bert(val_df)
dev_bert_emb = embed_by_bert(dev_df)
test_bert_emb = embed_by_bert(test_df)

X_train, y_train = featurize(dev_bert_emb)
X_val , y_val = featurize(val_bert_emb)
X_test, y_test = featurize(test_bert_emb)

train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = ClassifierDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
test_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

target_list = []
for _, t in train_dataset:
    target_list.append(t)
    
target_list = torch.tensor(target_list)
target_list = target_list[torch.randperm(len(target_list))]

class_count = [i for i in get_class_distribution(y_train).values()]
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
# print(class_weights)

class_weights_all = class_weights[target_list]

weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

EPOCHS = 300
BATCH_SIZE = 16
LEARNING_RATE = 0.0007
NUM_FEATURES = 2304
NUM_CLASSES = 3

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, sampler=weighted_sampler)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# print(model)

accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

# albert_train_preds = []

print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(X_train_batch)

        probs = nn.Softmax(dim=1)(y_train_pred)
        # preds = torch.max(probs, 1)[1]
        # albert_train_preds.append([float(probs[0][0]), float(probs[0][1]), float(probs[0][2])])
        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f}') 
    print(f'Val Loss: {val_epoch_loss/len(val_loader):.5f}')
    print(f'Train Acc: {train_epoch_acc/len(train_loader):.3f}')
    print(f'Val Acc: {val_epoch_acc/len(val_loader):.3f}')

end = time.time()
print(end - start)

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] ▁ " he ▁admitted ▁making ▁four ▁trips ▁to ▁china ▁and ▁playing ▁golf ▁there . ▁he ▁also ▁admitted ▁that ▁z te ▁officials , ▁whom ▁he ▁says ▁are ▁his ▁golf ▁buddies , ▁hosted ▁and ▁paid ▁for ▁the ▁trips . ▁jose ▁de ▁ vene cia ▁iii , ▁son ▁of ▁house ▁speaker ▁jose ▁de ▁ vene cia ▁jr , ▁alleged ▁that ▁a bal os ▁offered ▁him ▁us $ 10 ▁million ▁to ▁withdraw ▁his ▁proposal ▁on ▁the ▁ n bn ▁project . " [SEP]
INFO:tensorflow:input_ids: 2 13 7 438 3297 544 222 9573 20 998 17 791 4034 80 9 24 67 3297 30 2052 591 3055 15 1368 24 898 50 33 4034 26530 15 2812 17 2192 26 14 9573 9 2712 121 13 10263 4321 1867 15 433 16 191 4687 2712 121 13 10263 4321 2000 15 5965 30 21 3817 759 1434 61 182 4403 1036 507 20 10664 33 5149 27 14 13 103 11954 669 9 7 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)



INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] ▁ " zo e ▁tel ford ▁ - - ▁played ▁the ▁police ▁officer ▁girlfriend ▁of ▁simon , ▁maggie . ▁dumped ▁by ▁simon ▁in ▁the ▁final ▁episode ▁of ▁series ▁1 , ▁after ▁he ▁slept ▁with ▁jenny , ▁and ▁is ▁not ▁seen ▁again . ▁phoebe ▁thomas ▁played ▁cheryl ▁cassidy , ▁pauline ' s ▁friend ▁and ▁also ▁a ▁year ▁11 ▁pupil ▁in ▁simon ' s ▁class . ▁dumped ▁her ▁boyfriend ▁following ▁simon ' s ▁advice ▁after ▁he ▁wouldn ' t ▁have ▁sex ▁with ▁her ▁but ▁later ▁realised ▁this ▁was ▁due ▁to ▁him ▁catching ▁crab s ▁off ▁her ▁friend ▁pauline . " [SEP]
INFO:tensorflow:input_ids: 2 13 7 2500 62 6757 1426 13 8 8 257 14 698 1069 5606 16 2214 15 8379 9 15754 34 2214 19 14 426 942 16 231 137 15 75 24 7132 29 8369 15 17 25 52 541 188 9 20336 831 257 22087 15603 15 18076 22 18 860 17 67 21 159 547 11990 19 2214 22 18 718 9 15754 36 6198 249 2214 22 18 4978 75 24 1265 22 38 57 1589 29 36 47 138 12514 48 23 397 20 61 9436 11895 1

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)



INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] ▁ " upon ▁their ▁acceptance ▁into ▁the ▁kon tin ent al ▁hockey ▁league , ▁ deh ner ▁left ▁finland ▁to ▁sign ▁a ▁contract ▁in ▁germany ▁with ▁eh c ▁m * n chen ▁of ▁the ▁del ▁on ▁june ▁18 , ▁2014 . ▁after ▁capturing ▁the ▁german ▁championship ▁with ▁the ▁m * n chen ▁team ▁in ▁2016 , ▁he ▁left ▁the ▁club ▁and ▁was ▁picked ▁up ▁by ▁fellow ▁del ▁side ▁eh c ▁wolf sburg ▁in ▁july ▁2016 . ▁former ▁nhl er ▁gary ▁su ter ▁and ▁olympic - med al ist ▁bob ▁su ter ▁are ▁ deh ner ' s ▁uncle s . ▁his ▁cousin ▁is ▁minnesota ▁wild ' s ▁alternate ▁captain ▁ryan ▁su ter . " [SEP]
INFO:tensorflow:input_ids: 2 13 7 20243 66 10156 77 14 5191 2864 2291 192 2184 278 15 13 22850 1031 225 5613 20 1676 21 1305 19 914 29 16177 150 307 2483 103 5549 16 14 1506 27 295 474 15 623 9 75 12859 14 548 599 29 14 307 2483 103 5549 173 19 690 15 24 225 14 288 17 23 2114 71 34 1657 1506 270 16177 150 2597 6333 19 313 690 9 336 6542 106

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)



Begin training.


Epoch 001: | Train Loss: 1.01068
Val Loss: 1.13986
Train Acc: 50.816
Val Acc: 37.885
Epoch 002: | Train Loss: 0.73724
Val Loss: 1.13685
Train Acc: 68.576
Val Acc: 41.630
Epoch 003: | Train Loss: 0.66022
Val Loss: 1.10881
Train Acc: 71.768
Val Acc: 47.797
Epoch 004: | Train Loss: 0.54083
Val Loss: 1.28731
Train Acc: 78.728
Val Acc: 42.511
Epoch 005: | Train Loss: 0.48243
Val Loss: 1.25074
Train Acc: 81.816
Val Acc: 45.815
Epoch 006: | Train Loss: 0.35657
Val Loss: 1.50399
Train Acc: 86.232
Val Acc: 43.392
Epoch 007: | Train Loss: 0.31185
Val Loss: 1.46139
Train Acc: 88.384
Val Acc: 41.410
Epoch 008: | Train Loss: 0.27957
Val Loss: 1.50612
Train Acc: 89.632
Val Acc: 45.154
Epoch 009: | Train Loss: 0.27235
Val Loss: 1.58395
Train Acc: 89.408
Val Acc: 48.238
Epoch 010: | Train Loss: 0.22746
Val Loss: 1.62464
Train Acc: 91.472
Val Acc: 46.256
Epoch 011: | Train Loss: 0.18200
Val Loss: 1.70142
Train Acc: 93.584
Val Acc: 48.678
Epoch 012: | Train Loss: 0.17573
Val Loss: 1.80624
Train Acc: 92.

In [38]:
print(end - start)

862.2819263935089


# Get Training Weights for Late fusion Model

In [40]:
y_train_pred_list = []
albert_train_preds = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in train_loader:
        X_batch = X_batch.to(device)
        y_train_pred = model(X_batch)
        # probs = nn.Softmax(dim=1)(y_test_pred)
        albert_train_preds.extend(y_train_pred.tolist())
        _, y_pred_tags = torch.max(y_train_pred, dim = 1)
        y_train_pred_list.append(y_pred_tags.cpu().numpy())
y_train_pred_list = [a.squeeze().tolist() for a in y_train_pred_list]

In [41]:
print(albert_train_preds)
len(albert_train_preds)

[[-10.492827415466309, 6.430241107940674, -3.3277971744537354], [5.339158058166504, -3.4060215950012207, -5.900485038757324], [17.838287353515625, -25.724138259887695, -10.603468894958496], [12.448497772216797, -10.349291801452637, -12.43375015258789], [-2.106818199157715, 6.903900623321533, -10.829522132873535], [7.565122127532959, -10.969015121459961, -4.610668182373047], [6.8547444343566895, -11.378409385681152, -3.5445556640625], [-5.800631999969482, 6.873424530029297, -6.498617172241211], [-10.647794723510742, 12.434248924255371, -9.806272506713867], [-8.547662734985352, -5.190649032592773, 4.990792274475098], [7.459901809692383, -7.553781509399414, -6.2474775314331055], [-7.5226359367370605, -8.286276817321777, 5.839081764221191], [-7.8271989822387695, 6.098625659942627, -4.342911720275879], [14.02364730834961, -21.20992660522461, -7.8726606369018555], [-13.94931697845459, 9.762503623962402, -6.009303569793701], [7.840201377868652, -8.074419975280762, -6.849146842956543], [-10.33

2000

# Get predictions for calculating accuracy, precision, recall

In [42]:
y_pred_list = []
albert_preds = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        albert_preds.append(y_test_pred)
        # print(y_test_pred)
        _, y_pred_tags = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tags.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [43]:
print(y_pred_list)

[1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 2, 0, 1, 0, 1, 0, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 2, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 2, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 2, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 2, 0, 0, 1, 0, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0, 1, 1, 2, 0, 0, 0, 1, 1, 0, 2, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 2, 2, 2, 0, 1, 2, 2, 1, 1, 1, 1, 0, 2, 0, 2, 1, 0, 0, 1, 1, 1, 

In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.53      0.53      0.53       918
           1       0.48      0.49      0.49       855
           2       0.18      0.15      0.16       227

    accuracy                           0.47      2000
   macro avg       0.39      0.39      0.39      2000
weighted avg       0.47      0.47      0.47      2000



In [45]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_list))

[[487 354  77]
 [347 422  86]
 [ 93  99  35]]


In [46]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_list))

[[487 354  77]
 [347 422  86]
 [ 93  99  35]]


In [47]:
y_pred_list = []
albert_test_preds = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        # probs = nn.Softmax(dim=1)(y_test_pred)
        # preds = torch.max(probs, 1)[1]
        # albert_preds.append([float(probs[0][0]), float(probs[0][1]), float(probs[0][2])])
        albert_test_preds.extend(y_test_pred.tolist())
        _, y_pred_tags = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tags.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

# Save weights which can be later used for Late Fusion

In [ ]:
import csv
with open('albert_train', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)   
    write.writerows(albert_train_preds)

In [ ]:
import csv
with open('albert_test', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)   
    write.writerows(albert_test_preds)

# Late Fusion

In [ ]:
bert_train_preds = pd.read_csv('bert_train', header=None)
bert_test_preds = pd.read_csv('bert_test', header=None)

In [ ]:
bert_train_preds = bert_train_preds.values.tolist()
bert_test_preds = bert_test_preds.values.tolist()

In [ ]:
print(len(bert_train_preds))
print(len(bert_test_preds))

2000
2000


In [ ]:
X_train = pd.concat([pd.DataFrame(bert_train_preds), pd.DataFrame(albert_train_preds)], axis=1)
print(X_train)

              0          1          2          0          1          2
0     10.974892 -15.270205  -9.919143 -28.730419  16.147238  -6.497130
1     -2.136844   5.622533 -10.146481  -6.819132  -7.989262   6.804869
2      2.823482  -5.436560  -3.657327  11.972151 -10.419395 -10.709352
3      0.711462   1.380729  -8.498755 -14.136111   8.243175  -3.647041
4    -16.189619  11.740435  -7.508574  -4.305228   4.872440  -4.817991
...         ...        ...        ...        ...        ...        ...
1995   4.828914  -3.032495  -8.888766 -15.400626  10.767344  -6.236086
1996  11.592005 -20.626102  -7.332647  -4.216844   7.032504  -8.534880
1997   4.994088  -3.008132  -8.720770   6.168057  -5.456748  -6.043819
1998  -7.031741   6.071712  -5.643101  -7.631064  -8.519232   7.317303
1999  -2.481954   5.758660  -9.271006 -11.970511   8.698915  -6.267465

[2000 rows x 6 columns]


In [ ]:
y_train

array([0, 0, 1, ..., 0, 0, 1])

In [ ]:
X_test = pd.concat([pd.DataFrame(bert_test_preds), pd.DataFrame(albert_test_preds)], axis=1)
print(X_test)

              0          1          2          0          1          2
0     -0.242388   3.252065  -9.946405  -0.040293   4.227477 -10.129842
1      1.852692   0.524064  -9.218081   4.259192  -2.240057  -6.048580
2      7.448842 -14.324432  -4.955216  -6.544798   5.980649  -6.065374
3     -3.606236   4.643353  -6.371252  -3.765430   1.118028  -1.601155
4     11.284456 -20.005501  -7.712418  13.809802 -20.022303  -6.271025
...         ...        ...        ...        ...        ...        ...
1995   6.131945 -14.653542  -2.575500  -3.173794   1.757222  -1.277446
1996   5.547508  -3.910492  -8.520976  -1.759112   3.166871  -5.398952
1997   4.254806  -9.940295  -2.156491  -2.709748   3.482681  -4.322409
1998   3.643074   0.885228 -13.602243   4.426612   0.717485 -10.951860
1999   3.382428  -2.586327  -5.810749  -0.137294   3.076669  -7.855352

[2000 rows x 6 columns]


In [ ]:
y_test

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
# X_train, y_train = featurize(pd.concat([val_bert_emb, dev_bert_emb]).sort_index().reset_index())
logit = LogisticRegression(C=1e-2, random_state=17, solver='lbfgs', 
                           multi_class='multinomial', max_iter=100,
                          n_jobs=4)
logit.fit(X_train, y_train)
# X_test, y_test = featurize(test_bert_emb)
logit_test_pred = logit.predict_proba(X_test)
log_loss(y_test, logit_test_pred)
A = logit_test_pred[:, 0]
B = logit_test_pred[:, 1]
N = logit_test_pred[:, 2]
preds = []
for i in range(2000):
    if A[i] > B[i] and A[i] > N[i]:
        preds.append(0)
    elif B[i] > A[i] and B[i] > N[i]:
        preds.append(1)
    else:
        preds.append(2)


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))

[[161 757   0]
 [143 712   0]
 [ 10 217   0]]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(preds,y_test))

              precision    recall  f1-score   support

           0       0.18      0.51      0.26       314
           1       0.83      0.42      0.56      1686
           2       0.00      0.00      0.00         0

    accuracy                           0.44      2000
   macro avg       0.34      0.31      0.27      2000
weighted avg       0.73      0.44      0.51      2000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15232439183533537431, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15703311680
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3159682282082030462
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [ ]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8384576144443995486, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15703311680
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1749055655976364235
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-0e435c7a-545e-84f2-3563-fa616fa547f4)


In [ ]:
!nvidia-smi

Sat May  1 02:16:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    34W / 250W |    349MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------